# Hubble WFC3 Grism Distortion

The goal of this notebook is to describe a proposed process of describing WFC3 Grism distortions with the Generalized World Coordinate System (GWCS). Due to time constraints, this work is currently incomplete due to working bugs, but a theoretical process is described below.

This package depends on a few packages and repositories:
* asdf
* astropy
* h5py
* grismconf
    * https://github.com/npirzkal/GRISMCONF
    * numpy, astropy and scipy should be installed if not automatically
* gwcs
* jwst
    * https://github.com/spacetelescope/jwst
* jwreftools
    * https://github.com/spacetelescope/jwreftools

## Background

aXe, the current HST Grism Analysis and Extraction Tool this AstroGrism project is intended to replace, describes the grism trace relative to the x-axis. This restricts aXe from becoming a general purpose grism tool, considering the JWST traces can be traced along the y-axis. As such, to try and leverage the work done by the JWST pipeline, we need to be able to describe the HST grism traces using GWCS, the method the JWST pipeline uses. 

Nadia Dencheva's jupyter notebook shows how the JWST pipeline ingests GRISM data: https://github.com/nden/documentation/blob/master/grisms/JWST_Grisms.ipynb

As in cell In[4], three reference files are provided, which I've copied below:
```python
reference_files = {
    'distortion': '/Users/dencheva/crds/references/jwst/nircam/jwst_nircam_distortion_0096.asdf',
    'specwcs': '/Users/dencheva/crds/references/jwst/nircam/jwst_nircam_specwcs_0009.asdf',
    'wavelengthrange': '/Users/dencheva/crds/references/jwst/nircam/jwst_nircam_wavelengthrange_0003.asdf'
    }
```

The specific one I'd bring attention to is the 'specwcs' file. This file is where the trace polynomials and models, described in astropy models, are described.
```python
specwcs = asdf.open(reference_files['specwcs']).tree
displ = specwcs['displ']
dispx = specwcs['dispx']
dispy = specwcs['dispy']
invdispl = specwcs['invdispl']
invdispx = specwcs['invdispx']
invdispy = specwcs['invdispy']
orders = specwcs['orders']
```

These are the models that eventually are fed into GWCS:
```python
gdetector = cf.Frame2D(name='grism_detector', 
                       axes_order=(0, 1),
                       unit=(u.pix, u.pix))
det2det = NIRCAMForwardRowGrismDispersion(orders,
                                          lmodels=displ,
                                          xmodels=invdispx,
                                          ymodels=dispy)
det2det.inverse = NIRCAMBackwardGrismDispersion(orders,
                                                lmodels=invdispl,
                                                xmodels=dispx,
                                                ymodels=dispy)

grism_pipeline = [(gdetector, det2det)]
...
from gwcs import WCS
wcsobj = WCS(grism_pipeline)
```

So if we can generate these specwcs files for WFC3, then we'd be golden! So let's take a look at how this file is generated. It is generated in the jwreftools.nircam.nircam.grism.reffiles.create_grism_specwcs method:
https://github.com/spacetelescope/jwreftools/blob/master/jwreftools/nircam/nircam_grism_reffiles.py
```python
def create_grism_specwcs(conffile="",
                         pupil=None,
                         module=None,
                         author="STScI",
                         history="",
                         outname=None):
    """
    Create an asdf reference file to hold Grism C (column) or Grism R (rows)
    configuration information, no sensativity information is included
    Note: The orders are named alphabetically, i.e. Order A, Order B
    There are also sensativity fits files which are tables of wavelength,
    sensativity, and error. These are specified in the conffile but will
    not be read in and saved in the output reference file for now.
    It's possible they may be included in the future, either here or as
    a separate reference files. Their use here would be to help define the
    min and max wavelengths which set the extent of the dispersed trace on
    the grism image. Convolving the sensitiviy file with the filter throughput
    allows one to calculate the wavelength of minimum throughput which defines
    the edges of the trace.
    direct_filter is not specified because it assumes that the wedge
    information (wx,wy) is included in the conf file in one of the key-value
    pairs, where the key includes the beam designation
     this reference file also contains the polynomial model which is
     appropriate for the coefficients which are listed.
     wavelength = DISPL(order,x0,y0,t)
     dx = DISPX(order,x0,y0,t)
     dy = DISPY(order,x0,y0,t)
     t = INVDISPX(order,x0,y0,dx)
     t = INVDISPY(order,x0,y0,dy)
     t = INVDISL(order,x0,y0, wavelength)
    Parameters
    ----------
    conffile : str
        The text file with configuration information, formatted as aXe expects
    pupil : str
        Name of the grism the conffile corresponds to
        Taken from the conffile name if not specified
    module : str
        Name of the Nircam module
        Taken from the conffile name if not specified
    author : str
        The name of the author
    history : str
        A comment about the refrence file to be saved with the meta information
    outname : str
        Output name for the reference file
    Returns
    -------
    fasdf : asdf.AsdfFile(jwst.datamodels.NIRCAMGrismModel)
    """
```

This method effectively reads in a 'conffile' and extracts certain information from it. This 'conffile' is the core of generating this 'specwcs' file. What are these 'conffiles'?

GRISMCONF, built by Nor Pirzkal and Russell Ryan, is a package that describes the grism traces via a set of parametric equations, using a change of variables, from "x" or "y" to "t". GRISMCONF uses a catalog of configuration files for each instrument and filter. These configurations can be found here:
* https://github.com/npirzkal/GRISM_WFC3
* https://github.com/npirzkal/GRISM_NIRCAM
* https://github.com/npirzkal/GRISM_NIRISS

According to a discussion I had with Nadia Dencheva, *the GRISMCONF configuration files are the same ones used by the JWST pipeline to generate the 'specwcs' files!* So it should be simple to just plug in a GRISM_WFC3 configuration file and go through the reduction from there! GRISMCONF should be the interface for us to bridge the gap between the HST Grism files and the JWST GRISM pipeline

## Pseudocode/Working Theory

In [1]:
import pathlib
import tempfile
from urllib.parse import urlparse
from urllib.request import urlretrieve

import asdf
from astropy.io import fits

from HST.hst_grism_reffiles import create_tsgrism_wavelengthrange, create_grism_specwcs

### Generating Reference Files
There are two primary reference files we need, as mentioned above: the `wavelengthrange` and `specwcs` reference files. As we generate them below, we'll append them to this blank dictionary we'll feed into the GRISM pipeline later:

In [2]:
reference_files = dict()

#### Wavelength Range Reference File
The first of the two requires no information from the data itself. It is an instrument specific reference file that helps determine where the geometric cutouts should happen based on a prespecified sensitivity value. These values were provided in a slack conversation from Nor. Let's generate it:

In [3]:
# Generate Wavelengthrange Reference File

wavelengthrange_filename = "WFC3_wavelengthrange.asdf"
create_tsgrism_wavelengthrange(outname=wavelengthrange_filename)

#Confirm output file location
wav_ref_path = pathlib.Path(wavelengthrange_filename).resolve()
if not wav_ref_path.exists():
    raise FileNotFoundError("Can't find where the wavelengthrange reference file was saved")
reference_files['wavelengthrange'] = str(wav_ref_path)

/Users/dnguyen/duydir/hackenv/lib/python3.8/site-packages/jwst/datamodels/validate.py:35: ValidationWarning: While validating exposure the following error occurred:
'WFC3_TSGRISM' is not one of ['FGS_DARK', 'FGS_FOCUS', 'FGS_IMAGE', 'FGS_INTFLAT', 'FGS_SKYFLAT', 'FGS_ACQ1', 'FGS_ACQ2', 'FGS_FINEGUIDE', 'FGS_ID-IMAGE', 'FGS_ID-STACK', 'FGS_TRACK', 'MIR_4QPM', 'MIR_CORONCAL', 'MIR_DARKALL', 'MIR_DARKIMG', 'MIR_DARKMRS', 'MIR_FLATALL', 'MIR_FLATIMAGE', 'MIR_FLATIMAGE-EXT', 'MIR_FLATMRS', 'MIR_FLATMRS-EXT', 'MIR_IMAGE', 'MIR_LRS-FIXEDSLIT', 'MIR_LRS-SLITLESS', 'MIR_LYOT', 'MIR_MRS', 'MIR_TACONFIRM', 'MIR_TACQ', 'NIS_AMI', 'NIS_DARK', 'NIS_EXTCAL', 'NIS_FOCUS', 'NIS_IMAGE', 'NIS_LAMP', 'NIS_SOSS', 'NIS_TACQ', 'NIS_TACONFIRM', 'NIS_WFSS', 'NRC_CORON', 'NRC_DARK', 'NRC_FLAT', 'NRC_FOCUS', 'NRC_GRISM', 'NRC_IMAGE', 'NRC_WFSS', 'NRC_LED', 'NRC_WFSC', 'NRC_TACONFIRM', 'NRC_TACQ', 'NRC_TSGRISM', 'NRC_TSIMAGE', 'NRS_AUTOFLAT', 'NRS_AUTOWAVE', 'NRS_BRIGHTOBJ', 'NRS_CONFIRM', 'NRS_DARK', 'NRS_FIXEDS

#### Create Grism 'specwcs' file
The `specwcs` reference file is a little more complex. It actually contains the models that describe the traces of the grism itself. Here, we'll need some additional information. Firstly, we need to know which grism is being used. Each grism will disperse the field differently. We can get this from the Science Data itself. Let's download our working files from the AstroGrism Shared Data Repository

In [4]:
ib6o23rsq_flt_2_SPC_url = 'https://stsci.box.com/shared/static/2ks8o8q57kw0htlvsqzwbuw8no0qv9vs.fits'
#ib6o23rsq_flt_2_opt_SPC_url= 'https://stsci.box.com/shared/static/tr7f7iip75670um01y1qco520dygk5tm.fits'

tempdir = pathlib.Path(tempfile.gettempdir())
ib6o23rsq_flt_2_SPC_path = tempdir / "ib6o23rsq_flt_2_SPC.fits"
#ib6o23rsq_flt_2_opt_SPC_path = tempdir / "ib6o23rsq_flt_2_opt_SPC.fits"

try:
    print("Please wait, downloading HST WFC3 Data files...")
    if not ib6o23rsq_flt_2_SPC_path.is_file():
        urlretrieve(ib6o23rsq_flt_2_SPC_url, ib6o23rsq_flt_2_SPC_path)
    #if not ib6o23rsq_flt_2_opt_SPC_path.is_file():
    #    urlretrieve(ib6o23rsq_flt_2_opt_SPC_url, ib6o23rsq_flt_2_opt_SPC_path)
    print("Download Successful")
except:
    print("Failed to download files...")
    raise

Please wait, downloading HST WFC3 Data files...
Download Successful


The GRISM is specified in the fits header under the 'FILTER' keyword:

In [5]:
hdu1 = fits.open(ib6o23rsq_flt_2_SPC_path)
filter = hdu1[0].header['FILTER']
print("Filter is: " + str(filter))

Filter is: G141


Now that we know the correct grism, we can grab the corresponding GRISMCONF cofiguration file from Nor (there are also sensitivity files if those are necessary)

In [6]:
conf_url = {'G102': "https://raw.githubusercontent.com/npirzkal/GRISM_WFC3/master/IR/G102.conf",
            'G141': "https://raw.githubusercontent.com/npirzkal/GRISM_WFC3/master/IR/G141.conf"}
try: 
    print("Please wait, downloading HST GRISMCONF configuration files for WFC3 Grism: " + str(filter) + "...")
    conf_filepath = tempdir / (str(filter) + ".conf")
    if not conf_filepath.exists():
        urlretrieve(conf_url[str(filter)], conf_filepath)
    print("Download Successful")
except:
    print("Failed to download files...")
    raise

Please wait, downloading HST GRISMCONF configuration files for WFC3 Grism: G141...
Download Successful


Now we can extract the relevant information out of the GRISMCONF configuration files to create our 'specwcs' asdf file:

In [7]:
specwcs_filename = "WFC3_" + str(filter) + "_specwcs.asdf"
create_grism_specwcs(conffile=str(conf_filepath), pupil=filter, outname=specwcs_filename)

#Confirm output file location
specwcs_ref_path = pathlib.Path(specwcs_filename).resolve()
if not specwcs_ref_path.exists():
    raise FileNotFoundError("Can't find where the wavelengthrange reference file was saved")
reference_files['specwcs'] = str(specwcs_ref_path)

DEBUG:jwst.datamodels.util:Environmental "PASS_INVALID_VALUES" cannot be found. Using default value of "False".
DEBUG:jwst.datamodels.util:Environmental "STRICT_VALIDATION" cannot be found. Using default value of "False".


Pupil is G141

Reading /var/folders/93/k31j59sj2493rzfqxc8r5z0r000171/T/G141.conf  ...
Setting NAXIS = (1014, 1014)
Setting WEDGE_F098M = (-0.067, -0.191)
Setting WEDGE_F140W = (0.0, 0.0)
Setting WEDGE_F153M = (-0.063, 0.183)
Setting WEDGE_F139M = (-0.027, 0.048)
Setting WEDGE_F127M = (-0.048, 0.132)
Setting WEDGE_F128N = (0.057, 0.17)
Setting WEDGE_F130N = (0.05, 0.073)
Setting WEDGE_F132N = (0.044, -0.077)
Setting WEDGE_F126N = (-0.181, -0.21)
Setting WEDGE_F167N = (-0.113, 0.082)
Setting WEDGE_F164N = (-0.086, 0.202)
Setting WEDGE_F160W = (-0.053, 0.031)
Setting WEDGE_F125W = (0.037, -0.118)
Setting WEDGE_F110W = (0.12, -0.132)
Setting WEDGE_F105W = (0.068, 0.05)
Setting XRANGE_+1 = (-183, 1099)
Setting YRANGE_+1 = (-50, 1064)
Setting DISPL_+1_0 = 10000.0
Setting DISPL_+1_1 = 8000.0
Setting DISPX_+1_0 = (23.305297197184455, -0.002008805800026929, -0.0015918773882200696, -3.0389378125431904e-08, 2.1432087333320722e-07, -1.994516301400279e-07)
Setting DISPX_+1_1 = (177.81868253423985,

/Users/dnguyen/duydir/hackenv/lib/python3.8/site-packages/jwst/datamodels/validate.py:35: ValidationWarning: While validating name the following error occurred:
'WFC3' is not one of ['NIRCAM', 'NIRSPEC', 'MIRI', 'TFI', 'FGS', 'NIRISS', 'ANY', 'N/A']

Failed validating 'enum' in schema:
    {'$schema': 'http://stsci.edu/schemas/asdf-schema/0.1.0/asdf-schema',
     'enum': ['NIRCAM',
              'NIRSPEC',
              'MIRI',
              'TFI',
              'FGS',
              'NIRISS',
              'ANY',
              'N/A'],
     'fits_keyword': 'INSTRUME',
     'title': 'Instrument used to acquire the data',
     'type': 'string'}

On instance:
    'WFC3'
  warnings.warn(errmsg, ValidationWarning)
/Users/dnguyen/duydir/hackenv/lib/python3.8/site-packages/jwst/datamodels/validate.py:35: ValidationWarning: While validating type the following error occurred:
'IR_GRISM' is not one of ['FGS_DARK', 'FGS_FOCUS', 'FGS_IMAGE', 'FGS_INTFLAT', 'FGS_SKYFLAT', 'FGS_ACQ1', 'FGS_ACQ2', 'FG

/Users/dnguyen/duydir/astrogrism_sandbox/HST/wcs_ref_model.py:78: ValidationWarning: 
  warnings.warn(str(errmsg), ValidationWarning)


With this specwcs file, we can now extract our polynomial models as the JWST pipeline does:

In [8]:
specwcs = asdf.open(reference_files['specwcs']).tree
displ = specwcs['displ']
dispx = specwcs['dispx']
dispy = specwcs['dispy']
invdispl = specwcs['invdispl']
#invdispx = specwcs['invdispx']
#invdispy = specwcs['invdispy']
orders = specwcs['order']

### Create Grism Pipeline via Dispersion Models
With these polynomial trace models, we can now create the appropriate detector GRISM Dispersion Models and thereby create the necessary products fo the grism pipeline.

Obviously, the GrismDispersion classes will need to be written for HST WFC3

In [ ]:
from gwcs import coordinate_frames as cf
from astropy import units as u

from hst.transforms.models import WFC3ForwardRowGrismDispersion, WFC3BackwardGrismDispersion
#from jwst.transforms.models import NIRCAMForwardRowGrismDispersion, NIRCAMBackwardGrismDispersion

gdetector = cf.Frame2D(name='grism_detector', 
                       axes_order=(0, 1),
                       unit=(u.pix, u.pix))
det2det = WFC3ForwardRowGrismDispersion(orders,
                                        lmodels=displ,
                                        xmodels=invdispx,
                                        ymodels=dispy)
det2det.inverse = WFC3BackwardGrismDispersion(orders,
                                              lmodels=invdispl,
                                              xmodels=dispx,
                                              ymodels=dispy)

grism_pipeline = [(gdetector, det2det)]

### Create Image pipeline from dispersed grism image and reference files
The appropriate WFC3 analogs for the distortion models and filters' wavelength ranges will need to be obtained with the help of a WFC3 SME

In [ ]:
from hst import datamodels
from hst.assign_wcs import wfc3
#from jwst import datamodels
#from jwst.assign_wcs import nircam

# open the dispersed exposure
input_model = datamodels.open(str(ib6o23rsq_flt_2_SPC_path))

image_pipeline = wfc3.imaging(input_model, reference_files)
imagepipe = []
world = image_pipeline.pop()
for cframe, trans in image_pipeline:
    trans = trans & (Identity(2))
    imagepipe.append((cframe, trans))
imagepipe.append((world))
grism_pipeline.extend(imagepipe)

### Create GWCS object
Using the pipeline products above, with both image and grism pipelines, we can now create our WCS object

In [ ]:
from gwcs import WCS

wcsobj = WCS(grism_pipeline)

## Work to be done
This notebook was originally intended to be a full working example of how this process should work, but obviously the amount of work needed to actually accomplish this feat is much larger than anyone realistically imagined. That being said, hopefully this notebook adequately shows that we can get there, and outlines how we should get there. In summary, here's what needs to be done in order to actually get this notebook to be functional:
1. Gather the WFC3 distortion models from a WFC3 Subject Matter Expert
2. "Genericize" or make a WFC3-specific version of create_grism_specwcs
    * This should be fairly straight forward, considering the GRISMCONF input conffile for the WFC3 grisms already exists
3. Similarly, "Genericize" or make a WFC3-specific version of create_tsgrism_wavelengthrange
4. Create the WFC3 GRISM Dispersion Models to injest the grism trace models
    * Arguably, the center piece of the original purpose of JDAT-12
5. Create the WFC3 imaging model to apply the given distortions onto
    * From my perspective, this one has the highest "black-box, unknown score" associated with it